In [ ]:
client_id = "<client-id>"
client_secret = "<client-secret>"
tenant_id = '<tenant-id>'
storage_account_name = "tokyoolympicdatamaaz"
container_name = "raw-data"


configs = {
"fs.azure.account.auth.type": "OAuth",
"fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
"fs.azure.account.oauth2.client.id": client_id,
"fs.azure.account.oauth2.client.secret": client_secret,
"fs.azure.account.oauth2.client.endpoint": f"https://login.microsoftonline.com/{tenant_id}/oauth2/token"
}

mount_point = f"/mnt/{container_name}"
#print(mount_point)
try:
    dbutils.fs.mount(
         source=f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/",
         mount_point=mount_point,
         extra_configs=configs
     )
except Exception as e:
     print(f"Error occurred: {e}")

#List files to verify mount
display(dbutils.fs.ls("/mnt/raw-data"))

In [ ]:
spark

In [ ]:
athletes = spark.read.format("csv").option("header","true").option("inferSchema","true").load("/mnt/raw-data/Athletes (1).csv")
coaches = spark.read.format("csv").option("header","true").option("inferSchema","true").load("/mnt/raw-data/Coaches (1).csv")
entriesgender = spark.read.format("csv").option("header","true").option("inferSchema","true").load("/mnt/raw-data/EntriesGender (1).csv")
medals = spark.read.format("csv").option("header","true").option("inferSchema","true").load("/mnt/raw-data/Medals (1).csv")
teams = spark.read.format("csv").option("header","true").option("inferSchema","true").load("/mnt/raw-data/Teams (1).csv")

In [ ]:
# Verifying the data and the datatype using the printschema()

athletes.show()

In [ ]:
athletes.printSchema()

In [ ]:
coaches.show()

In [ ]:
coaches.printSchema()

In [ ]:
entriesgender.show()

In [ ]:
entriesgender.printSchema()

In [ ]:
# Typecasting columns into Integer Datatype

from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType, DoubleType, BooleanType, DateType

entriesgender = entriesgender.withColumn("Female",col("Female").cast(IntegerType()))\
    .withColumn("Male",col("Male").cast(IntegerType()))\
    .withColumn("Total",col("Total").cast(IntegerType()))

In [ ]:
entriesgender.printSchema()

In [ ]:
medals.show()

In [ ]:
medals.printSchema()

In [ ]:
teams.show()

In [ ]:
teams.printSchema()

In [ ]:
# Calculate the average number of entries by gender for each discipline
average_entries_by_gender = entriesgender.withColumn(
    'Avg_Female', entriesgender['Female'] / entriesgender['Total']
).withColumn(
    'Avg_Male', entriesgender['Male'] / entriesgender['Total']
)
average_entries_by_gender.show()

In [ ]:
athletes.repartition(1).write.mode("overwrite").option("header",'true').csv("/mnt/raw-data/transformed-data/athletes")

In [ ]:
coaches.repartition(1).write.mode("overwrite").option("header","true").csv("/mnt/raw-data/transformed-data/coaches")
entriesgender.repartition(1).write.mode("overwrite").option("header","true").csv("/mnt/raw-data/transformed-data/entriesgender")
medals.repartition(1).write.mode("overwrite").option("header","true").csv("/mnt/raw-data/transformed-data/medals")
teams.repartition(1).write.mode("overwrite").option("header","true").csv("/mnt/raw-data/transformed-data/teams")